In [ ]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, GRU, LSTM, Bidirectional, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
data = pd.read_csv("urdu-sentiment-corpus-v1 (2).tsv" , sep='\t')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   1000 non-null   object
 1   Class   999 non-null    object
dtypes: object(2)
memory usage: 15.8+ KB


In [ ]:
data.isna().sum()

Tweet    0
Class    1
dtype: int64

In [ ]:
data = data.dropna()

In [ ]:
data.isna().sum()

Tweet    0
Class    0
dtype: int64

In [ ]:
data["Class"].unique()

array(['P', 'N', 'O'], dtype=object)

In [ ]:
count_O = data["Class"].value_counts().get('O', 0)
print("Count of 'O':", count_O)

Count of 'O': 20


In [ ]:
#dropping rows with O
data = data[data["Class"] != 'O']

In [ ]:
data["Class"].unique()

array(['P', 'N'], dtype=object)

In [ ]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Tweet'])
sequences = tokenizer.texts_to_sequences(data['Tweet'])

In [ ]:
max_len = max([len(seq) for seq in sequences])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
X = pad_sequences(sequences, maxlen=max_len)
y = np.array([1 if label == 'P' else 0 for label in data['Class']])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
def create_model(model_type, num_layers, dropout_rate):
    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length=max_len))

    #Layers
    for _ in range(num_layers - 1):
        if model_type == 'RNN':
            model.add(SimpleRNN(64, return_sequences=True))
        elif model_type == 'GRU':
            model.add(GRU(64, return_sequences=True))
        elif model_type == 'LSTM':
            model.add(LSTM(64, return_sequences=True))
        elif model_type == 'BiLSTM':
            model.add(Bidirectional(LSTM(64, return_sequences=True)))
        model.add(Dropout(dropout_rate))

    # Final layer with return_sequences=False
    if model_type == 'RNN':
        model.add(SimpleRNN(64, return_sequences=False))
    elif model_type == 'GRU':
        model.add(GRU(64, return_sequences=False))
    elif model_type == 'LSTM':
        model.add(LSTM(64, return_sequences=False))
    elif model_type == 'BiLSTM':
        model.add(Bidirectional(LSTM(64, return_sequences=False)))
    model.add(Dropout(dropout_rate))

    # Using sigmoid
    model.add(Dense(1, activation='sigmoid'))

    # Compiling the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
hyperparameters = [
    {'num_layers': 2, 'dropout': 0.3},
    {'num_layers': 2, 'dropout': 0.7},
    {'num_layers': 3, 'dropout': 0.3},
    {'num_layers': 3, 'dropout': 0.7}
]

In [ ]:
results = []


for param in hyperparameters:
    for model_type in ['RNN', 'GRU', 'LSTM', 'BiLSTM']:
        model = create_model(model_type, param['num_layers'], param['dropout'])
        history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, verbose=0)
        y_pred = (model.predict(X_test) > 0.5).astype(int)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f_score = f1_score(y_test, y_pred)
        results.append({
            'Model': model_type,
            'Layers': param['num_layers'],
            'Dropout Rate': param['dropout'],
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F-score': f_score
        })

8/8 [==============================] - 4s 40ms/step


In [ ]:
evaluation = pd.DataFrame(results)
evaluation

,Model,Layers,Dropout Rate,Accuracy,Precision,Recall,F-score
0,RNN,2,0.3,0.453061,0.421488,0.443478,0.432203
1,GRU,2,0.3,0.591837,0.561983,0.591304,0.576271
2,LSTM,2,0.3,0.604082,0.561644,0.713043,0.628352
3,BiLSTM,2,0.3,0.608163,0.587156,0.556522,0.571429
4,RNN,2,0.7,0.546939,0.514925,0.600000,0.554217
5,GRU,2,0.7,0.595918,0.567797,0.582609,0.575107
6,LSTM,2,0.7,0.608163,0.579832,0.600000,0.589744
7,BiLSTM,2,0.7,0.595918,0.546512,0.817391,0.655052
8,RNN,3,0.3,0.497959,0.467213,0.495652,0.481013
9,GRU,3,0.3,0.600000,0.587629,0.495652,0.537736


In [ ]:
max = evaluation.loc[evaluation['F-score'].idxmax()]
print(max)

Model             BiLSTM
Layers                 2
Dropout Rate         0.7
Accuracy        0.595918
Precision       0.546512
Recall          0.817391
F-score         0.655052
Name: 7, dtype: object


In [ ]:
bilstm = evaluation[(evaluation['Model'] == 'BiLSTM') & (evaluation['Layers'] == 2) & (evaluation['Dropout Rate'] == 0.7)]


if not bilstm.empty:
    accuracy_bilstm = bilstm['Accuracy'].values[0]
    precision_bilstm = bilstm['Precision'].values[0]
    recall_bilstm = bilstm['Recall'].values[0]
    f_score_bilstm = bilstm['F-score'].values[0]

    print(f"Accuracy: {accuracy_bilstm}")
    print(f"Precision: {precision_bilstm}")
    print(f"Recall: {recall_bilstm}")
    print(f"F-Score: {f_score_bilstm}")

Accuracy: 0.5959183673469388
Precision: 0.5465116279069767
Recall: 0.8173913043478261
F-Score: 0.6550522648083624


In [ ]:
#GloVe embeddings
def load_glove_embeddings(embedding_file):
    embeddings_index = {}
    with open(embedding_file, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            embeddings = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = embeddings
        return embeddings_index

embedding_dim = 300
glove_embeddings = load_glove_embeddings('glove.6B.300d.txt')

#embedding matrix
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

#model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))

#compile it
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#train it
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)


y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy_glove = accuracy_score(y_test, y_pred)
precision_glove = precision_score(y_test, y_pred)
recall_glove = recall_score(y_test, y_pred)
f_score_glove = f1_score(y_test, y_pred)

print("Accuracy:", accuracy_glove)
print("Precision:", precision_glove)
print("Recall:", recall_glove)
print("F-score:", f_score_glove)

Epoch 1/10
10/10 [==============================] - 4s 108ms/step - loss: 0.6941 - accuracy: 0.4974 - val_loss: 0.6951 - val_accuracy: 0.4286
Epoch 2/10
10/10 [==============================] - 0s 30ms/step - loss: 0.6900 - accuracy: 0.5162 - val_loss: 0.6987 - val_accuracy: 0.4422
Epoch 3/10
10/10 [==============================] - 0s 28ms/step - loss: 0.6875 - accuracy: 0.5315 - val_loss: 0.7015 - val_accuracy: 0.4422
Epoch 4/10
10/10 [==============================] - 0s 30ms/step - loss: 0.6858 - accuracy: 0.5315 - val_loss: 0.7043 - val_accuracy: 0.4422
Epoch 5/10
10/10 [==============================] - 0s 29ms/step - loss: 0.6815 - accuracy: 0.5043 - val_loss: 0.7081 - val_accuracy: 0.4422
Epoch 6/10
10/10 [==============================] - 0s 28ms/step - loss: 0.6790 - accuracy: 0.5179 - val_loss: 0.7175 - val_accuracy: 0.4490
Epoch 7/10
10/10 [==============================] - 0s 29ms/step - loss: 0.6764 - accuracy: 0.5264 - val_loss: 0.7138 - val_accuracy: 0.5238
Epoch 8/10
1

In [ ]:
# Word2vec embeddings
from gensim.models import KeyedVectors

word2vec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

#embedding matrix
embedding_dim_wordvec = 300
embedding_matrix_wordvec = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    if word in word2vec_model:
        embedding_matrix_wordvec[i] = word2vec_model[word]

#model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim_wordvec, weights=[embedding_matrix_wordvec], input_length=max_len, trainable=False))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))

#compile it
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#train it
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)


y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy_wordvec = accuracy_score(y_test, y_pred)
precision_wordvec = precision_score(y_test, y_pred)
recall_wordvec = recall_score(y_test, y_pred)
f_score_wordvec = f1_score(y_test, y_pred)

print("Accuracy:", accuracy_wordvec)
print("Precision:", precision_wordvec)
print("Recall:", recall_wordvec)
print("F-score:", f_score_wordvec)

Epoch 1/10
10/10 [==============================] - 4s 110ms/step - loss: 0.6935 - accuracy: 0.4787 - val_loss: 0.6936 - val_accuracy: 0.4558
Epoch 2/10
10/10 [==============================] - 0s 42ms/step - loss: 0.6929 - accuracy: 0.5145 - val_loss: 0.6944 - val_accuracy: 0.4558
Epoch 3/10
10/10 [==============================] - 0s 41ms/step - loss: 0.6921 - accuracy: 0.5213 - val_loss: 0.6954 - val_accuracy: 0.4558
Epoch 4/10
10/10 [==============================] - 0s 41ms/step - loss: 0.6919 - accuracy: 0.5213 - val_loss: 0.6954 - val_accuracy: 0.4558
Epoch 5/10
10/10 [==============================] - 0s 31ms/step - loss: 0.6911 - accuracy: 0.5196 - val_loss: 0.6951 - val_accuracy: 0.4558
Epoch 6/10
10/10 [==============================] - 0s 31ms/step - loss: 0.6908 - accuracy: 0.5162 - val_loss: 0.6956 - val_accuracy: 0.4490
Epoch 7/10
10/10 [==============================] - 0s 31ms/step - loss: 0.6902 - accuracy: 0.5230 - val_loss: 0.6944 - val_accuracy: 0.4558
Epoch 8/10
1

In [ ]:
#FastText embeddings
def load_fasttext_embeddings(embedding_file):
    embeddings_index = {}
    with open(embedding_file, encoding='utf-8') as f:
        for line in f:
            values = line.rstrip().split(' ')
            word = values[0]
            embeddings = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = embeddings
    return embeddings_index

embedding_dim = 300
fasttext_embeddings = load_fasttext_embeddings('wiki-news-300d-1M.vec')

#embedding matrix
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = fasttext_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

#model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))

#compile it
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#train it
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy_fasttext = accuracy_score(y_test, y_pred)
precision_fasttext = precision_score(y_test, y_pred)
recall_fasttext = recall_score(y_test, y_pred)
f_score_fasttext = f1_score(y_test, y_pred)

print("Accuracy:", accuracy_fasttext)
print("Precision:", precision_fasttext)
print("Recall:", recall_fasttext)
print("F-score:", f_score_fasttext)

Epoch 1/10
10/10 [==============================] - 4s 106ms/step - loss: 0.6964 - accuracy: 0.5026 - val_loss: 0.6908 - val_accuracy: 0.5850
Epoch 2/10
10/10 [==============================] - 0s 32ms/step - loss: 0.6898 - accuracy: 0.5639 - val_loss: 0.6926 - val_accuracy: 0.4490
Epoch 3/10
10/10 [==============================] - 0s 30ms/step - loss: 0.6895 - accuracy: 0.5179 - val_loss: 0.6938 - val_accuracy: 0.4626
Epoch 4/10
10/10 [==============================] - 0s 35ms/step - loss: 0.6867 - accuracy: 0.5383 - val_loss: 0.6902 - val_accuracy: 0.5102
Epoch 5/10
10/10 [==============================] - 0s 32ms/step - loss: 0.6838 - accuracy: 0.5468 - val_loss: 0.6922 - val_accuracy: 0.4558
Epoch 6/10
10/10 [==============================] - 0s 35ms/step - loss: 0.6819 - accuracy: 0.5639 - val_loss: 0.6873 - val_accuracy: 0.5306
Epoch 7/10
10/10 [==============================] - 0s 32ms/step - loss: 0.6820 - accuracy: 0.5281 - val_loss: 0.6889 - val_accuracy: 0.5442
Epoch 8/10
1

In [ ]:
#Elmo
import tensorflow as tf
import tensorflow_hub as hub

data['Class'] = data['Class'].replace({'P': 1, 'N': 0})


#loading the pre-trained ELMo model
elmo = hub.load("https://tfhub.dev/google/elmo/3")

#converting the Tweets into embeddings
X = elmo.signatures["default"](tf.constant(data['Tweet'].tolist()))["elmo"]

#model
model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True), input_shape=(None, 1024)),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.7),
    Dense(1, activation='sigmoid')
])

X_train, X_test, y_train, y_test = train_test_split(X.numpy(), data['Class'], test_size=0.2, random_state=42)

#compling it
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)


y_pred = (model.predict(X_test) > 0.5).astype(int)


accuracy_elmo = accuracy_score(y_test, y_pred)
precision_elmo  = precision_score(y_test, y_pred)
recall_elmo  = recall_score(y_test, y_pred)
f1_elmo  = f1_score(y_test, y_pred)


print("Accuracy:", accuracy_elmo)
print("Precision:", precision_elmo)
print("Recall:", recall_elmo)
print("F1 Score:", f1_elmo)


Epoch 1/10
10/10 [==============================] - 16s 669ms/step - loss: 0.7017 - accuracy: 0.5208 - val_loss: 0.7037 - val_accuracy: 0.4395
Epoch 2/10
10/10 [==============================] - 3s 290ms/step - loss: 0.6881 - accuracy: 0.5335 - val_loss: 0.6955 - val_accuracy: 0.4968
Epoch 3/10
10/10 [==============================] - 3s 289ms/step - loss: 0.6664 - accuracy: 0.6054 - val_loss: 0.7034 - val_accuracy: 0.5287
Epoch 4/10
10/10 [==============================] - 3s 293ms/step - loss: 0.6279 - accuracy: 0.6502 - val_loss: 0.8420 - val_accuracy: 0.4268
Epoch 5/10
10/10 [==============================] - 4s 403ms/step - loss: 0.6153 - accuracy: 0.6454 - val_loss: 0.8449 - val_accuracy: 0.4140
Epoch 6/10
10/10 [==============================] - 3s 317ms/step - loss: 0.5911 - accuracy: 0.6869 - val_loss: 0.7717 - val_accuracy: 0.4777
Epoch 7/10
10/10 [==============================] - 3s 298ms/step - loss: 0.5622 - accuracy: 0.7141 - val_loss: 0.9859 - val_accuracy: 0.4140
Epoch

In [ ]:
evaluation_data = {
    'Model': ['BiLSTM (Without Embedding)', 'BiLSTM GloVe', 'BiLSTM Word2Vec', 'BiLSTM FastText' , 'BiLSTM Elmo'],
    'Accuracy': [accuracy_bilstm, accuracy_glove, accuracy_wordvec, accuracy_fasttext ,accuracy_elmo],
    'Precision': [precision_bilstm, precision_glove, precision_wordvec, precision_fasttext,precision_elmo],
    'Recall': [recall_bilstm, recall_glove, recall_wordvec, recall_fasttext,recall_elmo],
    'F-score': [f_score_bilstm, f_score_glove, f_score_wordvec, f_score_fasttext,f1_elmo]
}


df = pd.DataFrame(evaluation_data)

df.set_index('Model', inplace=True)

print(df)

                            Accuracy  Precision    Recall   F-score
Model                                                              
BiLSTM (Without Embedding)  0.595918   0.546512  0.817391  0.655052
BiLSTM GloVe                0.469388   0.468354  0.965217  0.630682
BiLSTM Word2Vec             0.461224   0.464730  0.973913  0.629213
BiLSTM FastText             0.608163   0.600000  0.495652  0.542857
BiLSTM Elmo                 0.602041   0.597015  0.439560  0.506329
